In [1]:
import gitlab
import re
import pandas as pd
import json

In [5]:
config_path = os.path.join('resources', 'config.json')
with open(config_path, 'r') as f:
    config = json.load(f)
    token = config.get('token')
    host = config.get('host')
    cs_2 = config.get('cs_2')

def get_issues(pId): 
    gl = gitlab.Gitlab(host, private_token=token, api_version=4, ssl_verify=True)
    project = gl.projects.get(pId)
    issues = project.issues.list(all=True)
    pmids = dict(high=[], low=[])
    for issue in issues:
        try:
            description = issue.description

            description = description.split('#### `Low Level Results`')
            h_pmid_list = re.findall(r'ncbi\.nlm\.nih\.gov\/pubmed\/(\d*)', description[0])
            l_pmid_list = re.findall(r'ncbi\.nlm\.nih\.gov\/pubmed\/(\d*)', description[1])

            pmids['high'] += h_pmid_list
            pmids['low'] += l_pmid_list
        except:
            continue

    pmids = {x: list(set(y)) for x, y in pmids.items()}

    return pmids

In [6]:
cs_2_pmids = get_issues(cs_2)
with open(os.path.join('out', 'cs_2_predictions.json'), 'w') as f:
    json.dump(cs_2_pmids, f)